# CS4248 Project Notebook

In [8]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [10]:
column_names = ['category', 'text']

train = pd.read_csv('fulltrain.csv', names=column_names)
X_train = train['text']
y_train = train['category']
# test = pd.read_csv('balancedtest.csv', names=column_names)

In [148]:
X_train[5654]

"In an unprecedented and historic event Monday, the 'I Am Under 18' button, an Internet security device which if selected restricts access to websites featuring adult content, was clicked for the first time ever. 'I knew I could simply claim to be over 18 and continue onto my desired destination, but I also realized that I would have to live with that lie for the rest of my life,' said local resident Garrett Kinley, 17. 'I admit, I was curious to see what type of material I would find on www.juggworld.com, but that button was clearly placed there for a reason, and let's face it: 17 and three-quarters is not 18. I plan to return to the site three months from now, when I will be mature enough to handle its content.' Moments later, Kinley's friend Dave Gerrard, 17, pushed Kinley aside and clicked the 'I Am Over 18' button himself, at which point a tactical police unit broke down his bedroom door and arrested him. "

In [150]:
def regex(X):
    pattern = r'\S+(?<=http)\S+||\S+(?<=https)\S+||\S+(?<=www)\S+||<\S+>||<\S+'
    X = re.sub(pattern, '', X)
    return X

regex(X_train[5654])

"In an unprecedented and historic event Monday, the 'I Am Under 18' button, an Internet security device which if selected restricts access to websites featuring adult content, was clicked for the first time ever. 'I knew I could simply claim to be over 18 and continue onto my desired destination, but I also realized that I would have to live with that lie for the rest of my life,' said local resident Garrett Kinley, 17. 'I admit, I was curious to see what type of material I would find on  but that button was clearly placed there for a reason, and let's face it: 17 and three-quarters is not 18. I plan to return to the site three months from now, when I will be mature enough to handle its content.' Moments later, Kinley's friend Dave Gerrard, 17, pushed Kinley aside and clicked the 'I Am Over 18' button himself, at which point a tactical police unit broke down his bedroom door and arrested him. "

## Preprocessing
#### Current
1. Tokenize the text
2. Lowercasing
3. Remove punctuation
4. Remove stop words
5. Lemmatization

In [36]:
def preprocess_text(text, lemmatize=False, remove_stopwords=False, remove_punctuations=False):
    tokens = word_tokenize(text.lower())

    if lemmatize:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    if remove_stopwords:
        tokens = [token for token in tokens if token not in stop_words]
    if remove_punctuations:
        tokens = [token for token in tokens if token.isalpha()]

    return tokens